# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
from astropy import constants as const
import multiprocessing

import platypos
from platypos import Planet_LoFo14
from platypos import Planet_Ot20

In [2]:
sys.path.append("../multi_track/")
from multi_track import evolve_one_planet_along_one_track
from multi_track import evolve_one_planet
from multi_track.create_planet_chunks import create_file_structure_for_saving_results_and_return_planet_folder_pairs
from multi_track.create_planet_chunks import make_folder_planet_track_list
from multi_track.create_planet_chunks import create_arg_list_one_planet, create_arg_list_one_planet_along_one_track

# Create Planets

In [4]:
star = {
 'star_id': "dummy",
 'mass': 0.9853484910214,
 'radius': None,
 'age': 1.0,
 'L_bol': 3.4539847848609653e+33/const.L_sun.cgs.value,
 'Lx_age': 2.56047429431498e+30}
planet = {
 'distance': 0.013243682995988661,
 'metallicity': 'solarZ',
 'fenv': 16.392784596958197,
 'core_mass': 0.9532237058218218}
pl1 = Planet_LoFo14(star, planet)

star = {
 'star_id': 'star_age1.0_mass1.03',
 'mass': 1.027672583692369,
 'radius': None,
 'age': 1.0,
 'L_bol': 4.0868258335169855e+33/const.L_sun.cgs.value,
 'Lx_age': 3.029605844799301e+30}
planet = {
 'distance': 0.26568969027997524,
 'metallicity': 'solarZ',
 'fenv': 2.2181748204555483,
 'core_mass': 2.274310743090016}
pl2 = Planet_LoFo14(star, planet)

star = {
 'star_id': 'star_age1.0_mass1.05',
 'mass': 1.0528330000727544,
 'radius': None,
 'age': 1.0,
 'L_bol': 4.6964654232268237e+33/const.L_sun.cgs.value,
 'Lx_age': 3.481537916153693e+30}
planet = {
 'distance': 0.08122846066224176,
 'metallicity': 'solarZ',
 'fenv': 7.4452478880800514,
 'core_mass': 3.5298972937511093}
pl3 = Planet_LoFo14(star, planet)

for pl in [pl1, pl2, pl3]:
    pl.Lx_sat_info = "Tu15"

In [5]:
pl2.__dict__

{'star_id': 'star_age1.0_mass1.03',
 'mass_star': 1.027672583692369,
 'radius_star': None,
 'age': 1.0,
 'Lbol': 4.0868258335169855e+33,
 'Lx_age': 3.029605844799301e+30,
 'Lx_sat_info': 'Tu15',
 'distance': 0.26568969027997524,
 'metallicity': 'solarZ',
 'flux': 14.993585465287188,
 'has_evolved': False,
 'planet_id': 'dummy',
 'planet_info': 'Case 1 - artificial planet (with M_core & f_env)',
 'fenv': 2.2181748204555483,
 'core_mass': 2.274310743090016,
 'core_radius': 1.228039823926851,
 'mass': 2.325903345446851,
 'radius': 4.298862337222561}

# Create Tracks

In [7]:
age_star = pl1.age
Lxsat_info = "LxTu15"
t_final, initial_step_size, epsilon = 3000., 0.1, 0.1
K, beta = "no", "no"

# track parameters which are different between the diff. tracks: t_sat, dt_drop, Lx_drop_factor
track1 = {"t_sat": age_star, "dt_drop": 20., "Lx_drop_factor": 18.}
track1_1 = {"t_sat": 10, "dt_drop": 20., "Lx_drop_factor": 12.}
track2 = {"t_sat": 20, "dt_drop": 20., "Lx_drop_factor": 10.}
track3 = {"t_sat": 40., "dt_drop": 20., "Lx_drop_factor": 7.}
track4 = {"t_sat": 70., "dt_drop": 20., "Lx_drop_factor": 5.}
track5 = {"t_sat": 100., "dt_drop": 0., "Lx_drop_factor": 0.}
track6 = {"t_sat": 150., "dt_drop": 0., "Lx_drop_factor": 0.}
track7 = {"t_sat": 240., "dt_drop": 0., "Lx_drop_factor": 0.}
different_track_params_list = [track1, track1_1, track2, track3, track4, track5, track6, track7]

# Evolve planets along all tracks

In [8]:
list_planets = [pl1, pl2, pl3]
name = "test_run/"

if not os.path.isdir(os.getcwd() + "/" + name):
    os.makedirs(os.getcwd() + "/" + name)
else:
    os.system("rm -r " + os.getcwd() + "/" + name)
    os.makedirs(os.getcwd() + "/" + name)

path_save, folder_planet_list = create_file_structure_for_saving_results_and_return_planet_folder_pairs(curr_path=os.getcwd()+"/", folder_name=name, list_planets=list_planets)
print("\nPath for saving results:\n", path_save)

# create list of [folder, planet, track]-triplets
folder_planet_track_list =  make_folder_planet_track_list(folder_planet_list, different_track_params_list)

# for evolve_one_planet_along_one_track
arg_tuples = create_arg_list_one_planet_along_one_track(folder_planet_track_list, t_final, initial_step_size, epsilon, K, beta, path_save)

Folder test_run/ exists.

Path for saving results:
 /work2/lketzer/work/PythonProjects/multi_track/multi_track/example/test_run/


In [9]:
%%time

if __name__ == '__main__': # Allows for the safe importing of the main module

    cpu_count = multiprocessing.cpu_count()
    print("There are %d CPUs on this machine" % cpu_count)

    with multiprocessing.Pool(processes=cpu_count-1) as pool:
        pool.starmap(evolve_one_planet_along_one_track, arg_tuples)

There are 8 CPUs on this machine


FileNotFoundError: [Errno 2] File /work2/lketzer/work/PythonProjects/multi_track/multi_track/venv/lib/python3.7/site-packages/platypos-1.1.3-py3.7.egg/supplementary_files/Mamajek_MS.csv does not exist: '/work2/lketzer/work/PythonProjects/multi_track/multi_track/venv/lib/python3.7/site-packages/platypos-1.1.3-py3.7.egg/supplementary_files/Mamajek_MS.csv'

In [ ]:
# %%time

# # evolve without multiprocessing
# for i, pl in enumerate([pl1, pl2, pl3]):
#     folder = "planet" + str(i+1)
#     if not os.path.isdir(os.getcwd() + "/" + folder):
#         os.makedirs(os.getcwd() + "/" + folder)
#         path_save = os.getcwd() + "/" + folder + "/"
#     else:
#         os.system("rm -r " + os.getcwd() + "/" + folder)
#         os.makedirs(os.getcwd() + "/" + folder)
#         path_save = os.getcwd() + "/" + folder + "/"
    
#     for track in different_track_params_list:
#         pl.Lx_sat_info = "Tu15"
#         folder_planet_track_list = [folder, pl, track]
#         evolve_one_planet_along_one_track(folder_planet_track_list, t_final, initial_step_size, epsilon, "no", "no", path_save)